In [1]:
import sys
sys.path.append('../src')
from models import *
from strategies import *
from custom_datasets import *
import numpy as np
np.random.seed(0)
import tqdm

import pytorch_lightning as pl
import segmentation_models_pytorch as smp
import os
import torch
torch.cuda.empty_cache()
import torch.nn as nn

from PIL import Image
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from torch.utils.data import Dataset
import pandas as pd

import time
import json
import wandb
from unet_model import *
from os.path import expanduser
from dbscan import DBScan, Similarities

/home/jovyan/my-conda-envs/myenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
home = expanduser("~")
main_path = home+"/Active-Learning-Segmentation/"
dataframes_path = main_path + "data/dataframes/"
sam_path = main_path + "sam/sam_vit_h_4b8939.pth"
notebooks_path = main_path + "scripts/notebooks/"
expirements_path = main_path+"expirements/"

In [3]:
# df_name = "brain_df"
# train_df = pd.read_csv(dataframes_path+"brain_df_train.csv")
# test_df = pd.read_csv(dataframes_path+"brain_df_test.csv")

df_name = "lung_tumor_df"
train_df = pd.read_csv(dataframes_path+"lung_df_train.csv")
test_df = pd.read_csv(dataframes_path+"lung_df_test.csv")

# df_name = "lunar_df"
# train_df = pd.read_csv(dataframes_path+"lunar_df_train.csv")
# test_df = pd.read_csv(dataframes_path+"lunar_df_test.csv")

In [4]:
len(test_df)

2930

In [5]:
len(train_df)

6836

In [6]:
# with open("brain_params.json", "r") as f:
#     params = json.load(f)
with open("lung_params.json", "r") as f:
    params = json.load(f)

# with open("lunar_params.json", "r") as f:
#     params = json.load(f)

In [7]:
params["n_epoch"]

35

In [8]:
if params["init_set_size"] == len(train_df):
    params["training_type"] = "no_active"
elif not params["use_sam"]:
    params["training_type"] = "no_sam"
elif params["voting"]:
    params["training_type"] = "voters"
else:
    params["training_type"]="withSAM_NoVoting"

if params["training_type"] == "no_active":
    if params["pre_trained"]:
        params["model_path"] = f'{notebooks_path}trained_models/{params["dataset"]}/no_active/pre_trained_Unet_{params["img_size"][0]}.pt'
    else:
        params["model_path"] = f'{notebooks_path}trained_models/{params["dataset"]}/no_active/not_pre_trained_Unet_{params["img_size"][0]}.pt'
else:
    params["model_path"] = f'{notebooks_path}trained_models/{params["dataset"]}/{params["training_type"]}/{params["init_set_size"]}'

if params["dropout"] and params["training_type"] != "voters":
    params["model_path"] = f'{params["model_path"]}_dropout'

params['test_set_size'] = len(test_df)
params['df'] = df_name
params['query_num'] = int(0.05 * params['init_set_size'])
if params['query_num'] == 0:
    params['query_num'] = 1
params["strategy"] = "MarginSampling"
notes = f'{params["training_type"]}_{params["init_set_size"]}'
if params["training_type"] == "voters":
    if params["similarity_check"]:
        params["model_path"] = f'{params["model_path"]}_dbscan'
        notes = f"{notes}_dbscan"
    if params["dropout"]:
        params["model_path"] = f'{params["model_path"]}_dropout'
        notes = f"{notes}_dropout"
    params["model_path"] = f'{params["model_path"]}/voters_{params["img_size"][0]}'
        

In [9]:
if params["pre_trained"]:
    notes = f"{notes}_pre_trained"
else:
    notes = f"{notes}_not_pre_trained"

In [10]:
wandb.init(
    # set the wandb project where this run will be logged
    project=params["dataset"],
    
    notes = notes,

    # track hyperparameters and run metadata
    config=params    
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: saleemfares1995-sf (thesis_fares). Use `wandb login --relogin` to force relogin


In [11]:
if params['use_sam']:
    sam = SAMOracle(checkpoint_path=sam_path, img_size=params["img_size"])
else:
    sam = None

In [12]:
trained_models_dir = notebooks_path+"trained_models"
if not os.path.exists(trained_models_dir):
    os.makedirs(trained_models_dir)
    
if params["pre_trained"]:
    model = smp.create_model('Unet', encoder_name='resnet34', in_channels=3, classes = 1)

    if not os.path.isfile(notebooks_path+"trained_models/shared_init_state_pre_trained.pt"):
        torch.save(model.state_dict(), notebooks_path+"trained_models/shared_init_state_pre_trained.pt")
        print("shared_init_state_pre_trained saved!")
    
    init_state_Unet = torch.load(notebooks_path+"trained_models/shared_init_state_pre_trained.pt")
    first_rd_Unet_path = notebooks_path+f"trained_models/shared_1st_state_pre_trained_{params['init_set_size']}.pt"
else:
    model = UNet(n_channels=3, n_classes=1, bilinear=True)

    if not os.path.isfile(notebooks_path+"trained_models/shared_init_state_not_trained.pt"):
        torch.save(model.state_dict(), notebooks_path+"trained_models/shared_init_state_not_trained.pt")
        print("shared_init_state_not_trained saved!")

    init_state_Unet = torch.load(notebooks_path+"trained_models/shared_init_state_not_trained.pt")
    tmp = "trained_models/shared_1st_state_not_trained"
    if params["dropout"]:
        tmp= f"{tmp}_dropout"
    first_rd_Unet_path = notebooks_path+f"{tmp}_{params['init_set_size']}.pt"
    
model.load_state_dict(init_state_Unet)

shared_init_state_not_trained saved!


<All keys matched successfully>

In [13]:
init_path = ""

if params["training_type"] == "voters":
    init_path = params["model_path"] + '_0/main_Unet.pt'

elif params["training_type"] == "withSAM_NoVoting":
    init_path = params["model_path"] + '/main_Unet_128_0.pt'
    
elif params["training_type"] == "no_sam":
    init_path = f'{params["model_path"]}/Active_{params["init_set_size"]}_{0}_no_sam_{params["img_size"][0]}_Unet.pt'
    
if len(init_path) > 0:
    if not os.path.isfile(init_path):
        init_dir = os.path.dirname(init_path)
        if not os.path.exists(init_dir):
            os.makedirs(init_dir)#
        torch.save(model.state_dict(), init_path)
        print(init_path, " saved!")
    # init_state_Unet = torch.load(init_path)
    # model.load_state_dict(init_state_Unet)
    


net = Net(model, params, device = torch.device("cuda"))

/home/jovyan/Active-Learning-Segmentation/scripts/notebooks/trained_models/Lunar_Rocky_Landscape/no_sam/500_dropout/Active_500_0_no_sam_128_Unet.pt  saved!


In [14]:
init_path

'/home/jovyan/Active-Learning-Segmentation/scripts/notebooks/trained_models/Lunar_Rocky_Landscape/no_sam/500_dropout/Active_500_0_no_sam_128_Unet.pt'

In [15]:
def get_data(handler, train_df, test_df):
    return Data(train_df["images"].to_list(), train_df["masks"].to_list(), test_df["images"].to_list(), test_df["masks"].to_list(), handler, img_size=params["img_size"], df=train_df, path= main_path+"/data/processed/", use_sam=params['use_sam'])

In [16]:
data = get_data(Handler, train_df, test_df)
data.initialize_labels(params["init_set_size"])

In [17]:
sim = Similarities()
# cosine_similarity, iou_score
db_scan = DBScan(similarities = [sim.iou_score],eps = 0.45, min_samples = 4)

In [18]:
strategy = MarginSampling(dataset=data, net=net, sam=sam, db_scan=db_scan, params=params)
params["strategy"] = "MarginSampling"

In [19]:
first_rd_Unet_path

'/home/jovyan/Active-Learning-Segmentation/scripts/notebooks/trained_models/shared_1st_state_not_trained_dropout_500.pt'

In [ ]:
print("Round 0")
rd = 1
logs =[]
main_path = ""
if params["training_type"] == "no_sam":
    main_path = f'{params["model_path"]}/Active_{params["init_set_size"]}_{rd}_no_sam_{params["img_size"][0]}_Unet.pt'

elif params["training_type"] == "voters":
    main_path = f'{params["model_path"]}_{rd}/main_Unet.pt'

elif params["training_type"] == "withSAM_NoVoting":
    main_path = f'{params["model_path"]}/main_Unet_{params["img_size"][0]}_{rd}.pt'



if not os.path.isfile(first_rd_Unet_path):
    strategy.train()
    torch.save(strategy.net.net.state_dict(), first_rd_Unet_path)
else:
    strategy.net.net.load_state_dict(torch.load(first_rd_Unet_path))
    
if len(main_path)>0:
    if not os.path.isfile(main_path):
        main_dir = os.path.dirname(main_path)
        if not os.path.exists(main_dir):
            os.makedirs(main_dir)
        torch.save(strategy.net.net.state_dict(), main_path)
        print("Saved : " + main_path)
    else:
        print(main_path)
        
else:
    main_path = f'{params["model_path"]}'
    main_dir = os.path.dirname(main_path)        
    if not os.path.exists(main_dir):
        os.makedirs(main_dir)
    torch.save(strategy.net.net.state_dict(), main_path)
    
logits, mask_gt = strategy.predict(data.get_test_data())
iou, dice_loss, cosine_similarity, eculidian_similarity, f1 = data.cal_test_metrics(logits, mask_gt )
# wandb.log({"iou_score" : iou, "dice_loss" : dice_loss, "cosine_similarity" : cosine_similarity, "eculidian_distance" : 1-eculidian_similarity, "f1_score" : f1})
logs.append(f"Round 0 testing metrics: iou_score = {iou:.2f}, dice_loss = {dice_loss:.2f}, cosine_similarity = {cosine_similarity:.2f}, eculidian_distance = {1-eculidian_similarity:.2f}, f1_score = {f1:.2f}")
print(logs[0])

for rd in range(1, params["rounds"]):
    print(f"Round {rd}")
    strategy.net.params['n_epoch']+=5
    # query
    print("Querying")
    query_idxs = strategy.query(params["query_num"])
    print(query_idxs)
    # update labels
    if params["use_sam"] and rd >= params["activate_sam_at_round"]:
        print("Updating with sam")
        if params["training_type"] == "voters":
            masks = strategy.update_voting(query_idxs, start_sam=True, use_predictor=params["use_predictor"], use_generator=params["use_generator"], round=rd)
        
        elif params["training_type"] == "withSAM_NoVoting":
            masks = strategy.update(query_idxs, start_sam=True, use_predictor=params["use_predictor"], use_generator=params["use_generator"], round=rd)
        # else:
            # masks = strategy.update_weighted_voting(query_idxs, start_sam=True, use_predictor=params["use_predictor"], use_generator=params["use_generator"], round=rd)
    else:
        print("Updating without sam")
        strategy.update(query_idxs)
    
    print("Reset and train")
    if params["training_type"] == "no_sam":
        main_path = f'{params["model_path"]}/Active_{params["init_set_size"]}_{rd+1}_no_sam_{params["img_size"][0]}_Unet.pt'

    elif params["training_type"] == "voters":
        main_path = f'{params["model_path"]}_{rd+1}/main_Unet.pt'

    elif params["training_type"] == "withSAM_NoVoting":
        main_path = f'{params["model_path"]}/main_Unet_{params["img_size"][0]}_{rd+1}.pt'

    if not os.path.isfile(main_path):
        strategy.net.net.load_state_dict(init_state_Unet)
        strategy.train()
        main_dir = os.path.dirname(main_path)
        if not os.path.exists(main_dir):
            os.makedirs(main_dir)
        torch.save(strategy.net.net.state_dict(), main_path)
        print("Saved : " + main_path)
    else:
        strategy.net.net.load_state_dict(torch.load(main_path))
    # print("uncomment the block above")
    # strategy.net.net.load_state_dict(init_state_Unet)
    # strategy.train()    

    # calculate accuracy
    logits, maks_gt = strategy.predict(data.get_test_data())
    iou, dice_loss, cosine_similarity, eculidian_similarity, f1 = data.cal_test_metrics(logits, mask_gt )
    # wandb.log({"iou_score" : iou, "dice_loss" : dice_loss, "cosine_similarity" : cosine_similarity, "eculidian_distance" : 1-eculidian_similarity, "f1_score" : f1})
    logs.append(f"Round {rd} testing metrics: iou_score = {iou:.2f}, dice_loss = {dice_loss:.2f}, cosine_similarity = {cosine_similarity:.2f}, eculidian_distance = {1-eculidian_similarity:.2f}, f1_score = {f1:.2f}")
    print(logs[rd])
    
params['logs'] = logs
wandb.finish()

Round 0
